In [5]:
VIDEO_LENGTH = 3
REST_TIME = 0.75
SAVE_DIR = "Videos"
NUM_VIDEO = 1
VIDEO_SOURCE = 0



In [6]:
import os
import cv2
import numpy as np
import time

In [7]:
def show_rest_time(width, height, rest_time, video_num):
    # Create a black frame for the "Rest time" message
    rest_frame = 0 * np.ones((height, width, 3), np.uint8)

    # Set the text parameters
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = f"Rest Time "
    (text_width, text_height), _ = cv2.getTextSize(text, font, 1, 2)
    org = ((width - text_width) // 2, (height - text_height) // 2)
    font_scale = 1
    color = (255,255,255)
    thickness = 2

    # Display the "Rest time" message for the specified duration
    start_time = time.time()
    while time.time() - start_time < rest_time:
        remaining_time = int(rest_time - (time.time() - start_time))
        text_with_time = f"{text} - {remaining_time} seconds left"
        rest_frame_copy = rest_frame.copy()
        cv2.putText(rest_frame_copy, text_with_time, org, font, font_scale, color, thickness)
        cv2.imshow("Rest Time", rest_frame_copy)
        cv2.waitKey(500) 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            return


In [9]:
import cv2
import numpy as np
import mediapipe as mp
max_videos_per_class = 30  # Maximum videos to process per class
min_frames_required = 30  # Minimum frames required in each video
videos_to_skip = 3  #
mp_holistic = mp.solutions.holistic.Holistic(static_image_mode=False, min_detection_confidence=0.2, min_tracking_confidence=0.2)
mp_hands = mp.solutions.hands.Hands(static_image_mode=False, min_detection_confidence=0.2, min_tracking_confidence=0.2)

def apply_mediapipe_holistic(frame):
    # Create a black background
    global mp_holistic
    black_frame = frame.copy()
    black_frame[:] = (0, 0, 0)

    # Convert the frame to RGB (Mediapipe requires RGB input)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Holistic
    results = mp_holistic.process(frame_rgb)

    # Draw the pose landmarks with orange color
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            black_frame, results.pose_landmarks, mp.solutions.holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(255, 165, 0), thickness=1, circle_radius=1),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(255, 165, 0), thickness=1))

    # Draw the left hand landmarks with blue color
    if results.left_hand_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            black_frame, results.left_hand_landmarks, mp.solutions.holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 0, 255), thickness=1))

    # Draw the right hand landmarks with purple color
    if results.right_hand_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            black_frame, results.right_hand_landmarks, mp.solutions.holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(128, 0, 128), thickness=1, circle_radius=1),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(128, 0, 128), thickness=1))

    return black_frame



In [14]:
import cv2
import os
import numpy as np

def record_camera(num_videos=NUM_VIDEO, video_length=VIDEO_LENGTH, rest_time=REST_TIME, video_source=VIDEO_SOURCE):
    frames_list = []  # Initialize a list to store frames

    # Initialize the video capture
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error: Unable to access the video source.")
        return

    # Set video dimensions (adjust as needed)
    for i in range(num_videos):
        # Record video for video_length seconds
        start_time = cv2.getTickCount()
        while (cv2.getTickCount() - start_time) / cv2.getTickFrequency() < video_length:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to capture frame.")
                break
            processed_frame = apply_mediapipe_holistic(frame)
            frames_list.append(processed_frame)  # Append the frame to the list

            cv2.imshow(f'Recording', frame)

            # Allow quitting recording by pressing 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                return frames_list  # Return the list of frames

        if i < num_videos - 1:
            show_rest_time(int(cap.get(3)), int(cap.get(4)), rest_time, class_name, i+1)

    cap.release()
    cv2.destroyAllWindows()

    return frames_list[0]  # Return the list of frames


In [15]:
record_camera()

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)